<a href="https://colab.research.google.com/github/Tom-Jung/Tom-Jung/blob/main/API_DART_F.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
pip install dart-fss

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.5/147.5 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.1/201.1 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 MB 18.8 MB/s eta 0:00:00
  Attempting uninstall: termcolor
    Found existing installation: termcolor 2.5.0
    Uninstalling termcolor-2.5.0:
      Successfully uninstalled termcolor-2.5.0


In [5]:
import requests
import pandas as pd

# DART API 키 입력
API_KEY = "85006d20472a255d5eae25ee265b5ed1b78d1386"

# 재무제표 요청 URL
BASE_URL = "https://opendart.fss.or.kr/api/fnlttSinglAcntAll.json"

In [ ]:
def get_financial_statement(corp_code: str, year: str, fs_div: str = "CFS"):
    """
    DART API를 통해 재무제표 데이터를 가져오는 함수
    :param corp_code: DART 시스템의 고유 회사 코드
    :param year: 조회 연도 (예: 2022)
    :param fs_div: 재무제표 구분 (CFS: 연결, OFS: 개별)
    :return: DataFrame 형태의 재무제표 데이터
    """
    params = {
        "crtfc_key": API_KEY,
        "corp_code": corp_code,
        "bsns_year": year,
        "reprt_code": "110131",  # 11011: 사업보고서
        "fs_div": fs_div
    }

    # API 요청
    response = requests.get(BASE_URL, params=params)
    data = response.json()

    if data['status'] != '000':
        print(f"API 오류: {data['message']}")
        return None

    # JSON 데이터를 DataFrame으로 변환
    reports = data['list']
    df = pd.DataFrame(reports)

    # 주요 컬럼 확인 및 데이터 정리
    expected_columns = ['sj_nm', 'account_nm', 'thstrm_amount', 'frmtrm_amount', 'bfefrmtrm_amount']
    available_columns = [col for col in expected_columns if col in df.columns]

    if not available_columns:
        print("유효한 재무제표 데이터가 없습니다.")
        return None

    df = df[available_columns]

    # 컬럼 이름 매핑
    column_mapping = {
        'sj_nm': '재무제표명',
        'account_nm': '계정명',
        'thstrm_amount': '당기 금액',
        'frmtrm_amount': '전기 금액',
        'bfefrmtrm_amount': '전전기 금액'
    }
    df.rename(columns=column_mapping, inplace=True)

    # 결측값 및 데이터 정제
    df.replace('', '0', inplace=True)
    numeric_columns = ['당기 금액', '전기 금액', '전전기 금액']
    for col in numeric_columns:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col])

    return df

# 예제 실행: 삼성전자(00126380) 2022년 재무제표 가져오기
corp_code = "00126380"  # DART에서 제공하는 회사 코드
year = "2022"
financial_data = get_financial_statement(corp_code, year)

if financial_data is not None:
    print(financial_data)
    # 엑셀로 저장
    financial_data.to_excel("financial_statement.xlsx", index=False)

     재무제표명              계정명            당기 금액         전기 금액
0    재무상태표             유동자산  232369082000000  2.181632e+14
1    재무상태표         현금및현금성자산   48944982000000  3.903142e+13
2    재무상태표           단기금융상품   75121426000000  8.170899e+13
3    재무상태표      단기상각후원가금융자산    1755225000000  3.369034e+12
4    재무상태표  단기당기손익-공정가치금융자산      53056000000  4.075700e+10
..     ...              ...              ...           ...
169  자본변동표             기말자본    -652689000000           NaN
170  자본변동표             기말자본  306391870000000           NaN
171  자본변동표             기말자본    8899049000000           NaN
172  자본변동표             기말자본  315290919000000           NaN
173  자본변동표             기말자본  301743152000000           NaN

[174 rows x 4 columns]


In [ ]:
# 손익계산서의 매출액, 매출원가, 판관비, 영업손익만 추출

# 재무제표 요청 URL
BASE_URL = "https://opendart.fss.or.kr/api/fnlttSinglAcntAll.json"

def get_filtered_financial_statement(corp_code: str, year: str, fs_div: str = "CFS"):
    """
    손익계산서에서 매출액, 매출원가, 판관비, 영업손익만 추출하는 함수
    """
    params = {
        "crtfc_key": API_KEY,
        "corp_code": corp_code,
        "bsns_year": year,
        "reprt_code": "11011",  # 11011: 사업보고서
        "fs_div": fs_div
    }

    # API 요청
    response = requests.get(BASE_URL, params=params)
    data = response.json()

    if data['status'] != '000':
        print(f"API 오류: {data['message']}")
        return None

    # JSON 데이터를 DataFrame으로 변환
    reports = data['list']
    df = pd.DataFrame(reports)

    # 손익계산서 항목 필터링 조건
    target_accounts = ["매출액", "매출원가", "판매비와관리비", "영업이익"]

    # 필요한 컬럼만 추출
    if not df.empty:
        df = df[df['account_nm'].isin(target_accounts)]
        df = df[['sj_nm', 'account_nm', 'thstrm_amount', 'frmtrm_amount']]

        # 컬럼 이름 매핑
        df.columns = ['재무제표명', '계정명', '당기 금액', '전기 금액']

        # 결측값 및 데이터 정제
        df.replace('', '0', inplace=True)
        df[['당기 금액', '전기 금액']] = df[['당기 금액', '전기 금액']].apply(pd.to_numeric)

    return df

# 예제 실행: 삼성전자(00126380) 2022년 손익계산서 주요 항목 추출
corp_code = "00126380"  # DART에서 제공하는 회사 코드
year = "2022"
filtered_financial_data = get_filtered_financial_statement(corp_code, year)

if filtered_financial_data is not None:
    print(filtered_financial_data)
    # 엑셀로 저장
    filtered_financial_data.to_excel("income_statement_filtered.xlsx", index=False)

    재무제표명      계정명            당기 금액            전기 금액
54  손익계산서     매출원가  190041770000000  166411342000000
56  손익계산서  판매비와관리비   68812960000000   61559601000000
57  손익계산서     영업이익   43376630000000   51633856000000


In [ ]:
import requests
import pandas as pd
import xml.etree.ElementTree as ET
import os

# API 키 입력
API_KEY = "85006d20472a255d5eae25ee265b5ed1b78d1386"

# 기업 코드 요청 URL
CORP_CODE_URL = f"https://opendart.fss.or.kr/api/corpCode.xml?crtfc_key={API_KEY}"
FINANCIAL_URL = "https://opendart.fss.or.kr/api/fnlttSinglAcntAll.json"

def download_corp_codes():
    """기업 코드 XML 파일 다운로드 및 파싱"""
    response = requests.get(CORP_CODE_URL)
    xml_path = "corpCode.xml"

    # XML 파일 저장
    with open(xml_path, "wb") as file:
        file.write(response.content)

    # XML 파싱
    tree = ET.parse(xml_path)
    root = tree.getroot()

    # 외감 기업 코드 및 이름 추출
    corp_list = []
    for corp in root.findall('list'):
        corp_code = corp.find('corp_code').text
        corp_name = corp.find('corp_name').text
        stock_code = corp.find('stock_code').text

        # 외감기업(stock_code 없는 기업 포함)
        if stock_code is None or stock_code.strip() == "":
            corp_list.append((corp_code, corp_name))

    return corp_list

def get_financial_data(corp_code: str, year: str):
    """특정 기업의 재무제표 데이터 수집"""
    params = {
        "crtfc_key": API_KEY,
        "corp_code": corp_code,
        "bsns_year": year,
        "reprt_code": "11011",  # 사업보고서
        "fs_div": "CFS"  # 연결재무제표
    }

    response = requests.get(FINANCIAL_URL, params=params)
    data = response.json()

    if data['status'] != '000':
        print(f"기업 코드 {corp_code} - API 오류: {data['message']}")
        return None

    # 손익계산서 필터링 항목
    target_accounts = ["매출액", "매출원가", "판매비와관리비", "영업이익"]
    reports = data.get('list', [])
    df = pd.DataFrame(reports)

    if not df.empty:
        df = df[df['account_nm'].isin(target_accounts)]
        df = df[['sj_nm', 'account_nm', 'thstrm_amount', 'frmtrm_amount']]
        df.columns = ['재무제표명', '계정명', '당기 금액', '전기 금액']
        df.replace('', '0', inplace=True)
        df[['당기 금액', '전기 금액']] = df[['당기 금액', '전기 금액']].apply(pd.to_numeric, errors='coerce').fillna(0)
        return df

    return None

def collect_all_financials(year: str):
    """모든 외감 기업의 재무제표 데이터를 수집하여 엑셀로 저장"""
    corp_list = download_corp_codes()
    all_data = []

    for corp_code, corp_name in corp_list:
        print(f"기업 {corp_name}({corp_code}) 재무제표 수집 중...")
        df = get_financial_data(corp_code, year)
        if df is not None:
            df['기업명'] = corp_name
            all_data.append(df)

    # 전체 데이터 통합
    if all_data:
        final_df = pd.concat(all_data, ignore_index=True)
        final_df.to_excel(f"financial_statements_{year}.xlsx", index=False)
        print(f"{year}년도 외감기업 재무제표 수집 완료!")
    else:
        print("수집된 데이터가 없습니다.")

# 실행: 모든 외감기업의 2022년 재무제표 수집
collect_all_financials("2022")


ParseError: not well-formed (invalid token): line 1, column 2 (<string>)

In [ ]:
import requests
import pandas as pd
import xml.etree.ElementTree as ET
import io

# API 키 입력
API_KEY = "85006d20472a255d5eae25ee265b5ed1b78d1386"

# 기업 코드 요청 URL
CORP_CODE_URL = f"https://opendart.fss.or.kr/api/corpCode.xml?crtfc_key={API_KEY}"

def download_corp_codes():
    """기업 코드 XML 파일 다운로드 및 파싱"""
    response = requests.get(CORP_CODE_URL)

    # 응답이 XML인지 확인
    if response.headers['Content-Type'] != 'application/xml':
        print("응답 형식이 XML이 아닙니다. 응답 내용:")
        print(response.text)
        return []

    # XML 파싱
    try:
        tree = ET.ElementTree(ET.fromstring(response.content))
        root = tree.getroot()

        # 외감 기업 코드 및 이름 추출
        corp_list = []
        for corp in root.findall('list'):
            corp_code = corp.find('corp_code').text
            corp_name = corp.find('corp_name').text
            stock_code = corp.find('stock_code').text

            # 외감기업 (stock_code 없는 기업 포함)
            if stock_code is None or stock_code.strip() == "":
                corp_list.append((corp_code, corp_name))

        return corp_list
    except ET.ParseError as e:
        print("XML 파싱 오류:", e)
        return []

# 테스트 실행
corp_list = download_corp_codes()
if corp_list:
    print(f"{len(corp_list)}개의 기업 코드가 수집되었습니다.")
else:
    print("기업 코드 수")


In [ ]:
import requests
import pandas as pd
import xml.etree.ElementTree as ET
import io

# API 키 설정
API_KEY = "85006d20472a255d5eae25ee265b5ed1b78d1386"

# API URL 설정
CORP_CODE_URL = f"https://opendart.fss.or.kr/api/corpCode.xml?crtfc_key={API_KEY}"
FINANCIAL_URL = "https://opendart.fss.or.kr/api/fnlttSinglAcntAll.json"

def download_corp_codes():
    """외감기업 코드를 다운로드 및 파싱"""
    response = requests.get(CORP_CODE_URL)

    # 인코딩 설정 및 XML 읽기
    response.encoding = response.apparent_encoding
    xml_data = response.content.decode(response.encoding)

    # XML 파싱
    root = ET.fromstring(xml_data)

    # 외감 기업 코드 및 이름 추출
    corp_list = []
    for corp in root.findall('list'):
        corp_code = corp.find('corp_code').text
        corp_name = corp.find('corp_name').text
        stock_code = corp.find('stock_code').text

        # 외감 기업(stock_code 없는 기업 포함)
        if stock_code is None or stock_code.strip() == "":
            corp_list.append((corp_code, corp_name))

    return corp_list

def get_financial_data(corp_code: str, year: str):
    """특정 기업의 손익계산서 주요 항목 수집"""
    params = {
        "crtfc_key": API_KEY,
        "corp_code": corp_code,
        "bsns_year": year,
        "reprt_code": "11011",  # 사업보고서
        "fs_div": "CFS"  # 연결재무제표
    }

    response = requests.get(FINANCIAL_URL, params=params)
    data = response.json()

    if data['status'] != '000':
        print(f"기업 코드 {corp_code} - API 오류: {data['message']}")
        return None

    # 손익계산서 필터링 항목
    target_accounts = ["매출액", "매출원가", "판매비와관리비", "영업이익"]
    reports = data.get('list', [])
    df = pd.DataFrame(reports)

    if not df.empty:
        df = df[df['account_nm'].isin(target_accounts)]
        df = df[['sj_nm', 'account_nm', 'thstrm_amount', 'frmtrm_amount']]
        df.columns = ['재무제표명', '계정명', '당기 금액', '전기 금액']
        df.replace('', '0', inplace=True)
        df[['당기 금액', '전기 금액']] = df[['당기 금액', '전기 금액']].apply(pd.to_numeric, errors='coerce').fillna(0)
        return df

    return None

def collect_all_financials(year: str):
    """모든 외감기업의 재무제표 데이터를 수집하여 엑셀로 저장"""
    corp_list = download_corp_codes()
    all_data = []

    for corp_code, corp_name in corp_list:
        print(f"기업 {corp_name}({corp_code}) 재무제표 수집 중...")
        df = get_financial_data(corp_code, year)
        if df is not None:
            df['기업명'] = corp_name
            all_data.append(df)

    # 전체 데이터 통합
    if all_data:
        final_df = pd.concat(all_data, ignore_index=True)
        final_df.to_excel(f"financial_statements_{year}.xlsx", index=False)
        print(f"{year}년도 외감기업 재무제표 수집 완료!")
    else:
        print("수집된 데이터가 없습니다.")

# 실행: 모든 외감기업의 2022년 재무제표 수집
collect_all_financials("2022")


TypeError: decode() argument 'encoding' must be str, not None

In [ ]:
import requests
import pandas as pd
import xml.etree.ElementTree as ET

# API 키 설정
API_KEY = "85006d20472a255d5eae25ee265b5ed1b78d1386"

# API URL 설정
CORP_CODE_URL = f"https://opendart.fss.or.kr/api/corpCode.xml?crtfc_key={API_KEY}"
FINANCIAL_URL = "https://opendart.fss.or.kr/api/fnlttSinglAcntAll.json"

def download_corp_codes():
    """외감기업 코드를 다운로드 및 파싱"""
    response = requests.get(CORP_CODE_URL)

    # 인코딩 설정: 'utf-8'을 기본으로 설정
    encoding = response.apparent_encoding if response.apparent_encoding else 'utf-8'
    xml_data = response.content.decode(encoding)

    # XML 파싱
    root = ET.fromstring(xml_data)

    # 외감 기업 코드 및 이름 추출
    corp_list = []
    for corp in root.findall('list'):
        corp_code = corp.find('corp_code').text
        corp_name = corp.find('corp_name').text
        stock_code = corp.find('stock_code').text

        # 외감기업 (stock_code 없는 기업 포함)
        if stock_code is None or stock_code.strip() == "":
            corp_list.append((corp_code, corp_name))

    return corp_list

def get_financial_data(corp_code: str, year: str):
    """특정 기업의 손익계산서 주요 항목 수집"""
    params = {
        "crtfc_key": API_KEY,
        "corp_code": corp_code,
        "bsns_year": year,
        "reprt_code": "11011",  # 사업보고서
        "fs_div": "CFS"  # 연결재무제표
    }

    response = requests.get(FINANCIAL_URL, params=params)
    data = response.json()

    if data['status'] != '000':
        print(f"기업 코드 {corp_code} - API 오류: {data['message']}")
        return None

    # 손익계산서 필터링 항목
    target_accounts = ["매출액", "매출원가", "판매비와관리비", "영업이익"]
    reports = data.get('list', [])
    df = pd.DataFrame(reports)

    if not df.empty:
        df = df[df['account_nm'].isin(target_accounts)]
        df = df[['sj_nm', 'account_nm', 'thstrm_amount', 'frmtrm_amount']]
        df.columns = ['재무제표명', '계정명', '당기 금액', '전기 금액']
        df.replace('', '0', inplace=True)
        df[['당기 금액', '전기 금액']] = df[['당기 금액', '전기 금액']].apply(pd.to_numeric, errors='coerce').fillna(0)
        return df

    return None

def collect_all_financials(year: str):
    """모든 외감기업의 재무제표 데이터를 수집하여 엑셀로 저장"""
    corp_list = download_corp_codes()
    all_data = []

    for corp_code, corp_name in corp_list:
        print(f"기업 {corp_name}({corp_code}) 재무제표 수집 중...")
        df = get_financial_data(corp_code, year)
        if df is not None:
            df['기업명'] = corp_name
            all_data.append(df)

    # 전체 데이터 통합
    if all_data:
        final_df = pd.concat(all_data, ignore_index=True)
        final_df.to_excel(f"financial_statements_{year}.xlsx", index=False)
        print(f"{year}년도 외감기업 재무제표 수집 완료!")
    else:
        print("수집된 데이터가 없습니다.")

# 실행: 모든 외감기업의 2022년 재무제표 수집
collect_all_financials("2022")


UnicodeDecodeError: 'utf-8' codec can't decode byte 0x8c in position 10: invalid start byte

In [11]:
import requests
import pandas as pd
import xml.etree.ElementTree as ET
import zipfile
import io

API_KEY = "85006d20472a255d5eae25ee265b5ed1b78d1386"

CORP_CODE_URL = f"https://opendart.fss.or.kr/api/corpCode.xml?crtfc_key={API_KEY}"
FINANCIAL_URL = "https://opendart.fss.or.kr/api/fnlttSinglAcntAll.json"

def download_corp_codes():
    """기업 코드 파일 다운로드 후 압축 해제 및 XML 파싱"""
    response = requests.get(CORP_CODE_URL)

    if response.status_code == 200:
        with zipfile.ZipFile(io.BytesIO(response.content)) as z:
            file_name = z.namelist()[0]
            with z.open(file_name) as xml_file:
                xml_data = xml_file.read().decode("utf-8")

        # XML 파싱
        root = ET.fromstring(xml_data)

        # 외감 기업 코드 및 이름 추출
        corp_list = []
        for corp in root.findall('list'):
            corp_code = corp.find('corp_code').text
            corp_name = corp.find('corp_name').text
            stock_code = corp.find('stock_code').text

            # 외감기업 (stock_code 없는 기업 포함)
            if stock_code is None or stock_code.strip() == "":
                corp_list.append((corp_code, corp_name))

        return corp_list
    else:
        print(f"기업 코드 다운로드 오류: {response.status_code}")
        return []

def get_financial_data(corp_code: str, year: str):
    """특정 기업의 손익계산서 주요 항목 수집"""
    params = {
        "crtfc_key": API_KEY,
        "corp_code": corp_code,
        "bsns_year": year,
        "reprt_code": "11011",  # 사업보고서
        "fs_div": "CFS"  # 연결재무제표
    }

    response = requests.get(FINANCIAL_URL, params=params)
    data = response.json()

    if data['status'] != '000':
        print(f"기업 코드 {corp_code} - API 오류: {data['message']}")
        return None

    # 손익계산서 필터링 항목
    target_accounts = ["매출액", "매출원가", "판매비와관리비", "영업이익"]
    reports = data.get('list', [])
    df = pd.DataFrame(reports)

    if not df.empty:
        df = df[df['account_nm'].isin(target_accounts)]
        df = df[['sj_nm', 'account_nm', 'thstrm_amount', 'frmtrm_amount']]
        df.columns = ['재무제표명', '계정명', '당기 금액', '전기 금액']
        df.replace('', '0', inplace=True)
        df[['당기 금액', '전기 금액']] = df[['당기 금액', '전기 금액']].apply(pd.to_numeric, errors='coerce').fillna(0)
        return df

    return None

def collect_all_financials(year: str):
    """모든 외감기업의 재무제표 데이터를 수집하여 엑셀로 저장"""
    corp_list = download_corp_codes()
    all_data = []

    for corp_code, corp_name in corp_list:
        print(f"기업 {corp_name}({corp_code}) 재무제표 수집 중...")
        df = get_financial_data(corp_code, year)
        if df is not None:
            df['기업명'] = corp_name
            all_data.append(df)

    # 전체 데이터 통합
    if all_data:
        final_df = pd.concat(all_data, ignore_index=True)
        final_df.to_excel(f"financial_statements_{year}.xlsx", index=False)
        print(f"{year}년도 외감기업 재무제표 수집 완료!")
    else:
        print("수집된 데이터가 없습니다.")

# 실행: 모든 외감기업의 2022년 재무제표 수집
collect_all_financials("2022")


기업 다코(00434003) 재무제표 수집 중...
기업 코드 00434003 - API 오류: 조회된 데이타가 없습니다.
기업 굿앤엘에스(00430964) 재무제표 수집 중...
기업 코드 00430964 - API 오류: 조회된 데이타가 없습니다.
기업 크레디피아제이십오차유동화전문회사(00388953) 재무제표 수집 중...
기업 코드 00388953 - API 오류: 조회된 데이타가 없습니다.
기업 연방건설산업(00179984) 재무제표 수집 중...
기업 코드 00179984 - API 오류: 조회된 데이타가 없습니다.
기업 브룩스피알아이오토메이션잉크(00420143) 재무제표 수집 중...
기업 코드 00420143 - API 오류: 조회된 데이타가 없습니다.
기업 매경아이비아이(00401111) 재무제표 수집 중...
기업 코드 00401111 - API 오류: 조회된 데이타가 없습니다.
기업 캐드뱅크(00435534) 재무제표 수집 중...


KeyboardInterrupt: 

In [10]:
corp_list

NameError: name 'corp_list' is not defined

In [12]:
import requests
import pandas as pd
import xml.etree.ElementTree as ET
import zipfile
import io
from concurrent.futures import ThreadPoolExecutor

API_KEY = "85006d20472a255d5eae25ee265b5ed1b78d1386"
CORP_CODE_URL = f"https://opendart.fss.or.kr/api/corpCode.xml?crtfc_key={API_KEY}"
FINANCIAL_URL = "https://opendart.fss.or.kr/api/fnlttSinglAcntAll.json"


def download_corp_codes():
    """기업 코드 파일 다운로드 후 압축 해제 및 XML 파싱"""
    response = requests.get(CORP_CODE_URL)

    if response.status_code == 200:
        with zipfile.ZipFile(io.BytesIO(response.content)) as z:
            file_name = z.namelist()[0]
            with z.open(file_name) as xml_file:
                xml_data = xml_file.read().decode("utf-8")

        # XML 파싱
        root = ET.fromstring(xml_data)
        corp_list = []
        for corp in root.findall('list'):
            corp_code = corp.find('corp_code').text
            corp_name = corp.find('corp_name').text
            stock_code = corp.find('stock_code').text

            # 외감기업 (stock_code 없는 기업 포함)
            if stock_code is None or stock_code.strip() == "":
                corp_list.append((corp_code, corp_name))

        return corp_list
    else:
        print(f"기업 코드 다운로드 오류: {response.status_code}")
        return []


def get_financial_data(corp_code: str, year: str):
    """특정 기업의 손익계산서 주요 항목 수집"""
    params = {
        "crtfc_key": API_KEY,
        "corp_code": corp_code,
        "bsns_year": year,
        "reprt_code": "11011",  # 사업보고서
        "fs_div": "CFS"  # 연결재무제표
    }

    response = requests.get(FINANCIAL_URL, params=params)
    try:
        data = response.json()
    except ValueError:
        print(f"기업 코드 {corp_code} - JSON 파싱 오류")
        return None

    if data.get('status') != '000':
        print(f"기업 코드 {corp_code} - API 오류: {data.get('message', '알 수 없는 오류')}")
        return None

    # 손익계산서 필터링 항목
    target_accounts = ["매출액", "매출원가", "판매비와관리비", "영업이익"]
    reports = data.get('list', [])
    df = pd.DataFrame(reports)

    if not df.empty:
        df = df[df['account_nm'].isin(target_accounts)]
        df = df[['sj_nm', 'account_nm', 'thstrm_amount', 'frmtrm_amount']]
        df.columns = ['재무제표명', '계정명', '당기 금액', '전기 금액']
        df.replace('', '0', inplace=True)
        df[['당기 금액', '전기 금액']] = df[['당기 금액', '전기 금액']].apply(pd.to_numeric, errors='coerce').fillna(0)
        return df

    return None


def collect_financial_data_for_corp(corp_tuple, year):
    """특정 외감기업 데이터 수집"""
    corp_code, corp_name = corp_tuple
    print(f"기업 {corp_name}({corp_code}) 재무제표 수집 중...")
    df = get_financial_data(corp_code, year)
    if df is not None:
        df['기업명'] = corp_name
    return df


def collect_all_financials(year: str):
    """모든 외감기업의 재무제표 데이터를 수집하여 엑셀로 저장"""
    corp_list = download_corp_codes()

    all_data = []
    with ThreadPoolExecutor(max_workers=10) as executor:
        results =


SyntaxError: invalid syntax (<ipython-input-12-24237b243961>, line 94)

In [24]:
import requests
import pandas as pd

API_KEY = "85006d20472a255d5eae25ee265b5ed1b78d1386"
FINANCIAL_URL = "https://opendart.fss.or.kr/api/fnlttSinglAcntAll.json"

def download_financial_statement(corp_code: str, year: str):
    """특정 기업의 재무제표 데이터 다운로드 및 저장"""
    params = {
        "crtfc_key": API_KEY,
        "corp_code": corp_code,
        "bsns_year": year,
        "reprt_code": "11011",  # 사업보고서
        "fs_div": "OFS"  # 개별 재무제표
    }

    response = requests.get(FINANCIAL_URL, params=params)
    data = response.json()

    if data.get('status') == '000':
        reports = data.get('list', [])
        if reports:
            df = pd.DataFrame(reports)
            # 주요 컬럼 필터링
            df = df[['sj_nm', 'account_nm', 'thstrm_amount', 'frmtrm_amount']]
            df.columns = ['재무제표명', '계정명', '당기 금액', '전기 금액']
            df.replace('', '0', inplace=True)
            df[['당기 금액', '전기 금액']] = df[['당기 금액', '전기 금액']].apply(pd.to_numeric, errors='coerce').fillna(0)

            # 엑셀 저장
            filename = f"financial_statement_{corp_code}_{year}.xlsx"
            df.to_excel(filename, index=False)
            print(f"재무제표가 {filename}에 저장되었습니다.")
        else:
            print(f"기업 코드 {corp_code}에 대한 재무제표 데이터가 없습니다.")
    else:
        print(f"재무제표 다운로드 오류: {data.get('message', '알 수 없는 오류')}")

# 예제 실행: 특정 기업 2022년도 재무제표 다운로드

corp_code = "00126380"  # 기업 코드 입력
year = "2018"
download_financial_statement(corp_code, year)


재무제표가 financial_statement_00126380_2018.xlsx에 저장되었습니다.


In [25]:
import requests
import pandas as pd
import xml.etree.ElementTree as ET
import zipfile
import io

API_KEY = "85006d20472a255d5eae25ee265b5ed1b78d1386"
CORP_CODE_URL = f"https://opendart.fss.or.kr/api/corpCode.xml?crtfc_key={API_KEY}"
FINANCIAL_URL = "https://opendart.fss.or.kr/api/fnlttSinglAcntAll.json"

def get_corp_codes():
    """기업 코드 리스트 다운로드 및 파싱"""
    response = requests.get(CORP_CODE_URL)
    corp_list = []

    if response.status_code == 200:
        with zipfile.ZipFile(io.BytesIO(response.content)) as z:
            file_name = z.namelist()[0]
            with z.open(file_name) as xml_file:
                xml_data = xml_file.read().decode("utf-8")

        root = ET.fromstring(xml_data)
        for corp in root.findall("list"):
            corp_code = corp.find("corp_code").text
            corp_name = corp.find("corp_name").text
            stock_code = corp.find("stock_code").text
            if corp_code and corp_name:  # 외감 기업도 포함
                corp_list.append((corp_code, corp_name))
    else:
        print(f"기업 코드 다운로드 오류: {response.status_code}")

    return corp_list

def get_financial_data(corp_code: str, corp_name: str, year: str):
    """특정 기업의 재무상태표 데이터 수집"""
    params = {
        "crtfc_key": API_KEY,
        "corp_code": corp_code,
        "bsns_year": year,
        "reprt_code": "11011",  # 사업보고서
        "fs_div": "CFS"  # 연결 재무제표 (개별 재무제표는 OFS)
    }

    response = requests.get(FINANCIAL_URL, params=params)
    data = response.json()

    if data.get('status') == '000' and 'list' in data:
        df = pd.DataFrame(data['list'])
        # 재무상태표만 필터링
        df = df[df['sj_nm'].str.contains('재무상태표|대차대조표', na=False)]
        if not df.empty:
            df = df[['sj_nm', 'account_nm', 'thstrm_amount', 'frmtrm_amount']]
            df.columns = ['재무제표명', '계정명', '당기 금액', '전기 금액']
            df.replace('', '0', inplace=True)
            df[['당기 금액', '전기 금액']] = df[['당기 금액', '전기 금액']].apply(pd.to_numeric, errors='coerce').fillna(0)
            df['기업명'] = corp_name
            return df
    return None

def collect_balance_sheets(year: str):
    """모든 기업의 재무상태표 데이터를 수집하여 엑셀로 저장"""
    corp_list = get_corp_codes()
    all_data = []

    for corp_code, corp_name in corp_list:
        print(f"기업 {corp_name}({corp_code}) 재무상태표 수집 중...")
        df = get_financial_data(corp_code, corp_name, year)
        if df is not None:
            all_data.append(df)

    if all_data:
        final_df = pd.concat(all_data, ignore_index=True)
        filename = f"balance_sheets_{year}.xlsx"
        final_df.to_excel(filename, index=False)
        print(f"{filename}에 모든 재무상태표 저장 완료!")
    else:
        print("수집된 재무상태표 데이터가 없습니다.")

# 실행: 모든 기업의 2022년 재무상태표 수집
collect_balance_sheets("2022")


기업 다코(00434003) 재무상태표 수집 중...
기업 굿앤엘에스(00430964) 재무상태표 수집 중...
기업 크레디피아제이십오차유동화전문회사(00388953) 재무상태표 수집 중...
기업 연방건설산업(00179984) 재무상태표 수집 중...
기업 브룩스피알아이오토메이션잉크(00420143) 재무상태표 수집 중...
기업 매경아이비아이(00401111) 재무상태표 수집 중...
기업 캐드뱅크(00435534) 재무상태표 수집 중...
기업 엠와이오피삼차유동화전문유한회사(00430186) 재무상태표 수집 중...
기업 엠와이오피이차유동화전문유한회사(00430201) 재무상태표 수집 중...
기업 엠와이오피일차유동화전문유한회사(00430210) 재무상태표 수집 중...
기업 포스미디어(00430229) 재무상태표 수집 중...
기업 축복할렐루야(00140432) 재무상태표 수집 중...
기업 한국전자화학(00426208) 재무상태표 수집 중...
기업 ConnachtCapitalMarketInvestmentLtd.(00433262) 재무상태표 수집 중...
기업 선진아이티(00433749) 재무상태표 수집 중...
기업 팀스코리아(00433785) 재무상태표 수집 중...
기업 에넥스하이테크(00196079) 재무상태표 수집 중...
기업 세이스텝바이스텝혼합형펀드(00435048) 재무상태표 수집 중...
기업 유리알파헷지채권혼합형펀드(00435057) 재무상태표 수집 중...
기업 한기술정보통신(00108843) 재무상태표 수집 중...
기업 다움종합건설(00443232) 재무상태표 수집 중...
기업 한국애치슨(00248293) 재무상태표 수집 중...
기업 고은상사(00455662) 재무상태표 수집 중...
기업 조흥종합건설(00149318) 재무상태표 수집 중...
기업 유니즈유통(00451347) 재무상태표 수집 중...
기업 창대화장품(00415105) 재무상태표 수집 중...
기업 문화종합건설(00420824) 재무상태표 수집 중...
기업

KeyboardInterrupt: 

In [26]:
import requests
import pandas as pd
import xml.etree.ElementTree as ET
import zipfile
import io

API_KEY = "85006d20472a255d5eae25ee265b5ed1b78d1386"
CORP_CODE_URL = f"https://opendart.fss.or.kr/api/corpCode.xml?crtfc_key={API_KEY}"
FINANCIAL_URL = "https://opendart.fss.or.kr/api/fnlttSinglAcntAll.json"

def get_corp_codes():
    """기업 코드 리스트 다운로드 및 파싱"""
    response = requests.get(CORP_CODE_URL)
    corp_list = []

    if response.status_code == 200:
        with zipfile.ZipFile(io.BytesIO(response.content)) as z:
            file_name = z.namelist()[0]
            with z.open(file_name) as xml_file:
                xml_data = xml_file.read().decode("utf-8")

        root = ET.fromstring(xml_data)
        for corp in root.findall("list"):
            corp_code = corp.find("corp_code").text
            corp_name = corp.find("corp_name").text
            if corp_code and corp_name:  # 외감 기업도 포함
                corp_list.append((corp_code, corp_name))
    else:
        print(f"기업 코드 다운로드 오류: {response.status_code}")

    return corp_list

def get_financial_data(corp_code: str, corp_name: str, year: str):
    """특정 기업의 재무상태표 데이터 수집"""
    params = {
        "crtfc_key": API_KEY,
        "corp_code": corp_code,
        "bsns_year": year,
        "reprt_code": "11011",  # 사업보고서
        "fs_div": "CFS"  # 연결 재무제표 (개별 재무제표는 OFS)
    }

    response = requests.get(FINANCIAL_URL, params=params)
    data = response.json()

    if data.get('status') == '000' and 'list' in data:
        df = pd.DataFrame(data['list'])
        # 재무상태표만 필터링
        df = df[df['sj_nm'].str.contains('재무상태표|대차대조표', na=False)]
        if not df.empty:
            df = df[['sj_nm', 'account_nm', 'thstrm_amount', 'frmtrm_amount']]
            df.columns = ['재무제표명', '계정명', '당기 금액', '전기 금액']
            df.replace('', '0', inplace=True)
            df[['당기 금액', '전기 금액']] = df[['당기 금액', '전기 금액']].apply(pd.to_numeric, errors='coerce').fillna(0)
            df['기업명'] = corp_name
            df['기업코드'] = corp_code  # 기업코드 추가
            return df
    return None

def collect_balance_sheets(year: str):
    """모든 기업의 재무상태표 데이터를 수집하여 엑셀로 저장"""
    corp_list = get_corp_codes()
    all_data = []

    for corp_code, corp_name in corp_list:
        print(f"기업 {corp_name}({corp_code}) 재무상태표 수집 중...")
        df = get_financial_data(corp_code, corp_name, year)
        if df is not None:
            all_data.append(df)

    if all_data:
        final_df = pd.concat(all_data, ignore_index=True)
        filename = f"balance_sheets_{year}.xlsx"
        final_df.to_excel(filename, index=False)
        print(f"{filename}에 모든 재무상태표 저장 완료!")
    else:
        print("수집된 재무상태표 데이터가 없습니다.")

# 실행: 모든 기업의 2022년 재무상태표 수집
collect_balance_sheets("2022")


기업 다코(00434003) 재무상태표 수집 중...
기업 굿앤엘에스(00430964) 재무상태표 수집 중...
기업 크레디피아제이십오차유동화전문회사(00388953) 재무상태표 수집 중...
기업 연방건설산업(00179984) 재무상태표 수집 중...
기업 브룩스피알아이오토메이션잉크(00420143) 재무상태표 수집 중...
기업 매경아이비아이(00401111) 재무상태표 수집 중...
기업 캐드뱅크(00435534) 재무상태표 수집 중...
기업 엠와이오피삼차유동화전문유한회사(00430186) 재무상태표 수집 중...
기업 엠와이오피이차유동화전문유한회사(00430201) 재무상태표 수집 중...
기업 엠와이오피일차유동화전문유한회사(00430210) 재무상태표 수집 중...
기업 포스미디어(00430229) 재무상태표 수집 중...
기업 축복할렐루야(00140432) 재무상태표 수집 중...
기업 한국전자화학(00426208) 재무상태표 수집 중...
기업 ConnachtCapitalMarketInvestmentLtd.(00433262) 재무상태표 수집 중...
기업 선진아이티(00433749) 재무상태표 수집 중...
기업 팀스코리아(00433785) 재무상태표 수집 중...
기업 에넥스하이테크(00196079) 재무상태표 수집 중...
기업 세이스텝바이스텝혼합형펀드(00435048) 재무상태표 수집 중...
기업 유리알파헷지채권혼합형펀드(00435057) 재무상태표 수집 중...
기업 한기술정보통신(00108843) 재무상태표 수집 중...
기업 다움종합건설(00443232) 재무상태표 수집 중...
기업 한국애치슨(00248293) 재무상태표 수집 중...
기업 고은상사(00455662) 재무상태표 수집 중...
기업 조흥종합건설(00149318) 재무상태표 수집 중...
기업 유니즈유통(00451347) 재무상태표 수집 중...
기업 창대화장품(00415105) 재무상태표 수집 중...
기업 문화종합건설(00420824) 재무상태표 수집 중...
기업

KeyboardInterrupt: 

In [31]:
import requests
import pandas as pd
import xml.etree.ElementTree as ET
import zipfile
import io

API_KEY = "85006d20472a255d5eae25ee265b5ed1b78d1386"
CORP_CODE_URL = f"https://opendart.fss.or.kr/api/corpCode.xml?crtfc_key={API_KEY}"
FINANCIAL_URL = "https://opendart.fss.or.kr/api/fnlttSinglAcntAll.json"

def get_corp_codes():
    """기업 코드 리스트 다운로드 및 파싱"""
    response = requests.get(CORP_CODE_URL)
    corp_list = []

    if response.status_code == 200:
        with zipfile.ZipFile(io.BytesIO(response.content)) as z:
            file_name = z.namelist()[0]
            with z.open(file_name) as xml_file:
                xml_data = xml_file.read().decode("utf-8")

        root = ET.fromstring(xml_data)
        for corp in root.findall("list"):
            corp_code = corp.find("corp_code").text
            corp_name = corp.find("corp_name").text
            if corp_code and corp_name:  # 외감 기업도 포함
                corp_list.append((corp_code, corp_name))
    else:
        print(f"기업 코드 다운로드 오류: {response.status_code}")

    return corp_list

def get_financial_data(corp_code: str, corp_name: str, year: str):
    """특정 기업의 재무상태표 데이터 수집"""
    params = {
        "crtfc_key": API_KEY,
        "corp_code": corp_code,
        "bsns_year": year,
        "reprt_code": "11011",  # 사업보고서
        "fs_div": "CFS"  # 연결 재무제표 (개별 재무제표는 OFS)
    }

    response = requests.get(FINANCIAL_URL, params=params)
    data = response.json()

    if data.get('status') == '000' and 'list' in data:
        df = pd.DataFrame(data['list'])
        # 재무상태표만 필터링
        df = df[df['sj_nm'].str.contains('재무상태표|대차대조표', na=False)]
        if not df.empty:
            df = df[['sj_nm', 'account_id', 'account_nm', 'thstrm_amount', 'frmtrm_amount']]
            df.columns = ['재무제표명', '계정코드', '계정명', '당기 금액', '전기 금액']
            df.replace('', '0', inplace=True)
            df[['당기 금액', '전기 금액']] = df[['당기 금액', '전기 금액']].apply(pd.to_numeric, errors='coerce').fillna(0)
            df['기업명'] = corp_name
            df['기업코드'] = corp_code
            return df
    return None

def collect_balance_sheets(year: str):
    """모든 기업의 재무상태표 데이터를 수집하여 엑셀로 저장"""
    corp_list = get_corp_codes()
    all_data = []

    for corp_code, corp_name in corp_list:
        print(f"기업 {corp_name}({corp_code}) 재무상태표 수집 중...")
        df = get_financial_data(corp_code, corp_name, year)
        if df is not None:
            all_data.append(df)

    if all_data:
        final_df = pd.concat(all_data, ignore_index=True)
        filename = f"balance_sheets_{year}.xlsx"
        final_df.to_excel(filename, index=False)
        print(f"{filename}에 모든 재무상태표 저장 완료!")
    else:
        print("수집된 재무상태표 데이터가 없습니다.")

# 실행: 모든 기업의 2022년 재무상태표 수집
collect_balance_sheets("2023")


기업 다코(00434003) 재무상태표 수집 중...
기업 굿앤엘에스(00430964) 재무상태표 수집 중...
기업 크레디피아제이십오차유동화전문회사(00388953) 재무상태표 수집 중...
기업 연방건설산업(00179984) 재무상태표 수집 중...
기업 브룩스피알아이오토메이션잉크(00420143) 재무상태표 수집 중...
기업 매경아이비아이(00401111) 재무상태표 수집 중...
기업 캐드뱅크(00435534) 재무상태표 수집 중...


KeyboardInterrupt: 

In [ ]:
import requests
import pandas as pd
import xml.etree.ElementTree as ET
import zipfile
import io

API_KEY = "85006d20472a255d5eae25ee265b5ed1b78d1386"
CORP_CODE_URL = f"https://opendart.fss.or.kr/api/corpCode.xml?crtfc_key={API_KEY}"
FINANCIAL_URL = "https://opendart.fss.or.kr/api/fnlttSinglAcntAll.json"

def get_corp_codes():
    """기업 코드 리스트 다운로드 및 파싱"""
    response = requests.get(CORP_CODE_URL)
    corp_list = []

    if response.status_code == 200:
        with zipfile.ZipFile(io.BytesIO(response.content)) as z:
            file_name = z.namelist()[0]
            with z.open(file_name) as xml_file:
                xml_data = xml_file.read().decode("utf-8")

        root = ET.fromstring(xml_data)
        for corp in root.findall("list"):
            corp_code = corp.find("corp_code").text
            corp_name = corp.find("corp_name").text
            if corp_code and corp_name:
                corp_list.append((corp_code, corp_name))
    else:
        print(f"기업 코드 다운로드 오류: {response.status_code}")

    return corp_list

def get_financial_data(corp_code: str, corp_name: str, year: str):
    """특정 기업의 재무상태표 데이터 수집"""
    params = {
        "crtfc_key": API_KEY,
        "corp_code": corp_code,
        "bsns_year": year,
        "reprt_code": "11011",  # 사업보고서
        "fs_div": "CFS"  # 연결 재무제표
    }

    response = requests.get(FINANCIAL_URL, params=params)
    data = response.json()

    if data.get('status') == '000' and 'list' in data:
        df = pd.DataFrame(data['list'])
        df = df[df['sj_nm'].str.contains('재무상태표|대차대조표', na=False)]
        if not df.empty:
            df = df[['sj_nm', 'account_id', 'account_nm', 'thstrm_amount', 'frmtrm_amount']]
            df.columns = ['재무제표명', '계정코드', '계정명', '당기 금액', '전기 금액']
            df.replace('', '0', inplace=True)
            df[['당기 금액', '전기 금액']] = df[['당기 금액', '전기 금액']].apply(pd.to_numeric, errors='coerce').fillna(0)
            df['기업명'] = corp_name
            df['기업코드'] = corp_code
            return df
    return None

def collect_balance_sheets(year: str):
    """모든 기업의 재무상태표 데이터를 수집하여 CSV로 저장"""
    corp_list = get_corp_codes()
    all_data = []

    for corp_code, corp_name in corp_list:
        print(f"기업 {corp_name}({corp_code}) 재무상태표 수집 중...")
        df = get_financial_data(corp_code, corp_name, year)
        if df is not None:
            all_data.append(df)

    if all_data:
        final_df = pd.concat(all_data, ignore_index=True)
        filename = f"balance_sheets_{year}.csv"
        final_df.to_csv(filename, index=False, encoding="utf-8-sig")
        print(f"{filename}에 모든 재무상태표 저장 완료!")
    else:
        print("수집된 재무상태표 데이터가 없습니다.")

# 실행: 모든 기업의 2022년 재무상태표 수집
collect_balance_sheets("2022")


기업 다코(00434003) 재무상태표 수집 중...
기업 굿앤엘에스(00430964) 재무상태표 수집 중...
기업 크레디피아제이십오차유동화전문회사(00388953) 재무상태표 수집 중...
기업 연방건설산업(00179984) 재무상태표 수집 중...
기업 브룩스피알아이오토메이션잉크(00420143) 재무상태표 수집 중...
기업 매경아이비아이(00401111) 재무상태표 수집 중...
기업 캐드뱅크(00435534) 재무상태표 수집 중...
기업 엠와이오피삼차유동화전문유한회사(00430186) 재무상태표 수집 중...
기업 엠와이오피이차유동화전문유한회사(00430201) 재무상태표 수집 중...
기업 엠와이오피일차유동화전문유한회사(00430210) 재무상태표 수집 중...
기업 포스미디어(00430229) 재무상태표 수집 중...
기업 축복할렐루야(00140432) 재무상태표 수집 중...
기업 한국전자화학(00426208) 재무상태표 수집 중...
기업 ConnachtCapitalMarketInvestmentLtd.(00433262) 재무상태표 수집 중...
기업 선진아이티(00433749) 재무상태표 수집 중...
기업 팀스코리아(00433785) 재무상태표 수집 중...
기업 에넥스하이테크(00196079) 재무상태표 수집 중...
기업 세이스텝바이스텝혼합형펀드(00435048) 재무상태표 수집 중...
기업 유리알파헷지채권혼합형펀드(00435057) 재무상태표 수집 중...
기업 한기술정보통신(00108843) 재무상태표 수집 중...
기업 다움종합건설(00443232) 재무상태표 수집 중...
기업 한국애치슨(00248293) 재무상태표 수집 중...
기업 고은상사(00455662) 재무상태표 수집 중...
기업 조흥종합건설(00149318) 재무상태표 수집 중...
기업 유니즈유통(00451347) 재무상태표 수집 중...
기업 창대화장품(00415105) 재무상태표 수집 중...
기업 문화종합건설(00420824) 재무상태표 수집 중...
기업